In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
bitcoin = pd.read_csv("../models/bitcoin_train.csv")
dogecoin = pd.read_csv("../models/dogecoin_train.csv")

In [32]:
bitcoin.columns.values.tolist()

['Unnamed: 0',
 'Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Daily_Change',
 'Daily_Change_Ind',
 'MACD',
 'PROC_3',
 'PROC_5',
 'PROC_10',
 'wpr',
 'sto_os',
 'goog_trend_score',
 'count',
 'compound_times_retweets',
 'likes_count',
 'neg_times_retweets',
 'pos_times_retweets',
 'replies_count',
 'retweets_count',
 'pos_weighted',
 'neg_weighted',
 'compound_weighted',
 'count_avg7',
 'count_daily_diff',
 'count_weekly_diff',
 'replies_count_avg7',
 'replies_count_daily_diff',
 'replies_count_weekly_diff',
 'retweets_count_avg7',
 'retweets_count_daily_diff',
 'retweets_count_weekly_diff',
 'likes_count_avg7',
 'likes_count_daily_diff',
 'likes_count_weekly_diff',
 'compound_weighted_avg7',
 'compound_weighted_daily_diff',
 'compound_weighted_weekly_diff',
 'pos_weighted_avg7',
 'pos_weighted_daily_diff',
 'pos_weighted_weekly_diff',
 'neg_weighted_avg7',
 'neg_weighted_daily_diff',
 'neg_weighted_weekly_diff',
 'Daily_Change_Perc',
 'RSI',
 'Weekly_Change',
 'Weekly_Chang

KeyError: 'ind'

In [4]:
feature_list = ['MACD','PROC_3', 'PROC_5', 'PROC_10', 'wpr', 'goog_trend_score', 'compound_weighted',\
                'neg_weighted', 'pos_weighted', 'count_avg7', 'count_daily_diff', 'count_weekly_diff',\
                'replies_count_avg7', 'replies_count_daily_diff', 'replies_count_weekly_diff', 'retweets_count_avg7',\
                'retweets_count_daily_diff', 'retweets_count_weekly_diff', 'likes_count_avg7',\
                'likes_count_daily_diff', 'likes_count_weekly_diff', 'compound_weighted_avg7', \
                'compound_weighted_daily_diff', 'compound_weighted_weekly_diff', 'pos_weighted_avg7', \
                'pos_weighted_daily_diff', 'pos_weighted_weekly_diff', 'neg_weighted_avg7', \
                'neg_weighted_daily_diff','neg_weighted_weekly_diff','Daily_Change_Perc', 'Weekly_Change_Perc',\
                'Daily_Change_Perc_Gold', 'Daily_Change_Perc_SP500', 'RSI', 'streak']

In [5]:
financial_features = ['MACD','PROC_3', 'PROC_5', 'PROC_10', 'wpr', 'RSI', 'Daily_Change_Perc',\
                      'Daily_Change_Perc_Gold', 'Daily_Change_Perc_SP500', 'RSI_Gold', 'RSI_SP500', 'streak']
social_features = ['goog_trend_score', 'compound_weighted',\
                'neg_weighted', 'pos_weighted', 'count_avg7', 'count_daily_diff', 'count_weekly_diff',\
                'replies_count_avg7', 'replies_count_daily_diff', 'replies_count_weekly_diff', 'retweets_count_avg7',\
                'retweets_count_daily_diff', 'retweets_count_weekly_diff', 'likes_count_avg7',\
                'likes_count_daily_diff', 'likes_count_weekly_diff', 'compound_weighted_avg7', \
                'compound_weighted_daily_diff', 'compound_weighted_weekly_diff', 'pos_weighted_avg7', \
                'pos_weighted_daily_diff', 'pos_weighted_weekly_diff', 'neg_weighted_avg7', \
                'neg_weighted_daily_diff','neg_weighted_weekly_diff',]

In [6]:
def get_corr_matrices(df, features, y):

    feature_corr = df[features].corr()
    columns = feature_corr.columns.values.tolist()
    
    def color_corr(x):
        colors = []
        for col in columns:
            if col == x.name:
                colors.append("background: gray")
            elif x[col] > 0.7 or x[col] < -0.7:
                colors.append("background: red")
            elif x[col] > 0.5 or x[col] < -0.5:
                colors.append("background: yellow")
            else: 
                colors.append("")
        return colors

    feature_corr = feature_corr.style.apply(color_corr, axis=1)
    label_corr_list = []

    for feature in features:
        corr = df[feature].corr(df[y])
        label_corr_list.append([feature, np.abs(corr)])
    
    label_corr_list.sort(reverse=True, key= lambda x : x[1])
    label_corr = pd.DataFrame(label_corr_list, columns = ['Feature', 'Corr'])

    return feature_corr, label_corr

In [7]:
bit_feat_corr, bit_label_corr = get_corr_matrices(bitcoin, financial_features, 'label')

In [8]:
bit_feat_corr

In [9]:
bit_label_corr

Feature      Corr
0                       RSI  0.076950
1                  RSI_Gold  0.070862
2                 RSI_SP500  0.065656
3   Daily_Change_Perc_SP500  0.046922
4                       wpr  0.030434
5    Daily_Change_Perc_Gold  0.027668
6                    streak  0.022834
7                    PROC_3  0.014206
8                      MACD  0.011540
9         Daily_Change_Perc  0.009212
10                  PROC_10  0.007456
11                   PROC_5  0.000029

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
rf = RandomForestClassifier()
rf.fit(bitcoin[financial_features], bitcoin['label'])
importances = rf.feature_importances_.tolist()

In [12]:
importances_df = pd.DataFrame(list(zip(financial_features, importances)), columns=["feature", "importance"])

In [13]:
importances_df.sort_values(by=['importance'], ascending=False)

feature  importance
0                      MACD    0.095001
5                       RSI    0.093806
10                RSI_SP500    0.092312
3                   PROC_10    0.090258
7    Daily_Change_Perc_Gold    0.087956
8   Daily_Change_Perc_SP500    0.087669
9                  RSI_Gold    0.085266
1                    PROC_3    0.084292
4                       wpr    0.083477
6         Daily_Change_Perc    0.082335
2                    PROC_5    0.077772
11                   streak    0.039857

In [17]:
social_feat_corr, social_label_corr = get_corr_matrices(bitcoin, social_features, 'label')

In [18]:
social_feat_corr

In [19]:
social_label_corr

Feature      Corr
0          compound_weighted_avg7  0.085825
1               pos_weighted_avg7  0.085565
2               compound_weighted  0.069644
3                    pos_weighted  0.066354
4          likes_count_daily_diff  0.061732
5         likes_count_weekly_diff  0.060719
6      retweets_count_weekly_diff  0.051222
7       retweets_count_daily_diff  0.043428
8              replies_count_avg7  0.039634
9               neg_weighted_avg7  0.038582
10                   neg_weighted  0.038193
11               goog_trend_score  0.029561
12       pos_weighted_weekly_diff  0.029184
13   compound_weighted_daily_diff  0.028931
14       neg_weighted_weekly_diff  0.021523
15        pos_weighted_daily_diff  0.021005
16      replies_count_weekly_diff  0.020083
17  compound_weighted_weekly_diff  0.019506
18               count_daily_diff  0.016360
19            retweets_count_avg7  0.013670
20                     count_avg7  0.010923
21       replies_count_daily_diff  0.009763
22              count_weekly_diff  0.009540
23        neg_weighted_daily_diff  0.008297
24               likes_count_avg7  0.002382

In [20]:
rf2 = RandomForestClassifier()
rf2.fit(bitcoin[social_features], bitcoin['label'])
importances2 = rf2.feature_importances_.tolist()

importances_df2 = pd.DataFrame(list(zip(social_features, importances2)), columns=["feature", "importance"])
importances_df2.sort_values(by=['importance'], ascending=False)

feature  importance
22              neg_weighted_avg7    0.045494
23        neg_weighted_daily_diff    0.045133
15        likes_count_weekly_diff    0.044065
17   compound_weighted_daily_diff    0.044011
19              pos_weighted_avg7    0.043701
16         compound_weighted_avg7    0.042932
14         likes_count_daily_diff    0.042926
20        pos_weighted_daily_diff    0.042436
6               count_weekly_diff    0.042220
9       replies_count_weekly_diff    0.042159
1               compound_weighted    0.042143
24       neg_weighted_weekly_diff    0.041988
3                    pos_weighted    0.040800
12     retweets_count_weekly_diff    0.039823
21       pos_weighted_weekly_diff    0.039663
11      retweets_count_daily_diff    0.039038
0                goog_trend_score    0.038015
8        replies_count_daily_diff    0.037227
18  compound_weighted_weekly_diff    0.037194
5                count_daily_diff    0.036424
10            retweets_count_avg7    0.036107
4                      count_avg7    0.035747
13               likes_count_avg7    0.033779
7              replies_count_avg7    0.033532
2                    neg_weighted    0.033443

In [49]:
features_to_try = ['MACD', 'RSI', 'PROC_3', 'Daily_Change_Perc_SP500', 'Daily_Change_Perc_Gold', \
                   'compound_weighted_avg7', 'likes_count_daily_diff', 'pos_weighted_avg7', 'retweets_count_weekly_diff',\
                  'goog_trend_score', 'RSI_Gold', 'RSI_SP500']

financial_features_final = ['RSI', 'RSI_Gold', 'RSI_SP500', 'Daily_Change_Perc_SP500',\
                            'Daily_Change_Perc_Gold','wpr', 'MACD', 'PROC_3']

sentiment_features_final = ['compound_weighted_avg7', 'compound_weighted', 'likes_count_daily_diff', \
                            'retweets_count_weekly_diff', 'likes_count_avg7', 'goog_trend_score',\
                           'neg_weighted', 'neg_weighted_avg7', 'pos_weighted_weekly_diff']

combined = financial_features_final + sentiment_features_final

print(combined)

['RSI', 'RSI_Gold', 'RSI_SP500', 'Daily_Change_Perc_SP500', 'Daily_Change_Perc_Gold', 'wpr', 'MACD', 'PROC_3', 'compound_weighted_avg7', 'compound_weighted', 'likes_count_daily_diff', 'retweets_count_weekly_diff', 'likes_count_avg7', 'goog_trend_score', 'neg_weighted', 'neg_weighted_avg7', 'pos_weighted_weekly_diff']


In [50]:
X = bitcoin[combined]
y = bitcoin['Daily_Change_Ind']

X = X.to_numpy()
y = y.to_numpy()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1234)
y_train = y_train.reshape(-1)

In [51]:
lr = LogisticRegression()

lr.fit(X_train, y_train)
y_hat = lr.predict(X_valid)

confusion = metrics.confusion_matrix(y_valid, y_hat)

accuracy = np.sum(np.diagonal(confusion))/len(y_valid)
print(accuracy)

0.6185897435897436
